In [1]:
%load_ext autoreload
%autoreload 2

# Introduction

## Setup

In [2]:
!pip install sklearn
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

You should consider upgrading via the '/home/katarina/git/rbp_encode_eclip/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/katarina/git/rbp_encode_eclip/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/katarina/git/rbp_encode_eclip/venv/bin/python -m pip install --upgrade pip' command.
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [3]:
import pandas as pd
from urllib import request
from tqdm import tqdm_notebook
from pathlib import Path
import yaml

## Download metadata file from Encode

In [4]:
!wget -O metadata.tsv "https://www.encodeproject.org/metadata/?status=released&internal_tags=ENCORE&assay_title=eCLIP&biosample_ontology.term_name=K562&biosample_ontology.term_name=HepG2&files.file_type=bed+narrowPeak&type=Experiment&files.analyses.status=released&files.preferred_default=true"

--2022-04-09 12:54:58--  https://www.encodeproject.org/metadata/?status=released&internal_tags=ENCORE&assay_title=eCLIP&biosample_ontology.term_name=K562&biosample_ontology.term_name=HepG2&files.file_type=bed+narrowPeak&type=Experiment&files.analyses.status=released&files.preferred_default=true
Resolving www.encodeproject.org (www.encodeproject.org)... 34.211.244.144
Connecting to www.encodeproject.org (www.encodeproject.org)|34.211.244.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/tsv]
Saving to: ‘metadata.tsv’

metadata.tsv            [   <=>              ] 125,40K   153KB/s    in 0,8s    

2022-04-09 12:55:00 (153 KB/s) - ‘metadata.tsv’ saved [128408]



In [5]:
metadata = pd.read_csv('metadata.tsv', sep='\t')

In [6]:
metadata

,File accession,File format,File type,File format type,Output type,File assembly,Experiment accession,Assay,Donor(s),Biosample term id,...,Genome annotation,Platform,Controlled by,File Status,s3_uri,File analysis title,File analysis status,Audit WARNING,Audit NOT_COMPLIANT,Audit ERROR
0,ENCFF311PHS,bed narrowPeak,bed,narrowPeak,peaks,GRCh38,ENCSR735HOK,eCLIP,/human-donors/ENCDO000AAC/,EFO:0001187,...,NaN,NaN,NaN,released,s3://encode-public/2018/04/29/dca28e48-1652-4c...,Lab custom GRCh38,released,antibody characterized with exemption,NaN,NaN
1,ENCFF203AWR,bed narrowPeak,bed,narrowPeak,peaks,GRCh38,ENCSR356MSW,eCLIP,/human-donors/ENCDO000AAD/,EFO:0002067,...,NaN,NaN,NaN,released,s3://encode-public/2018/04/29/2a02265d-be38-42...,Lab custom GRCh38,released,mixed read lengths,NaN,NaN
2,ENCFF890DJI,bed narrowPeak,bed,narrowPeak,peaks,GRCh38,ENCSR820UYE,eCLIP,/human-donors/ENCDO000AAC/,EFO:0001187,...,NaN,NaN,NaN,released,s3://encode-public/2018/04/29/b217e442-31c4-47...,Lab custom GRCh38,released,NaN,NaN,NaN
3,ENCFF484NSW,bed narrowPeak,bed,narrowPeak,peaks,GRCh38,ENCSR734ZHL,eCLIP,/human-donors/ENCDO000AAD/,EFO:0002067,...,NaN,NaN,NaN,released,s3://encode-public/2018/04/29/2bd059f7-b792-4e...,Lab custom GRCh38,released,NaN,NaN,NaN
4,ENCFF093FTO,bed narrowPeak,bed,narrowPeak,peaks,GRCh38,ENCSR202HKN,eCLIP,/human-donors/ENCDO000AAD/,EFO:0002067,...,NaN,NaN,NaN,released,s3://encode-public/2018/04/29/d1fb8b83-7d6a-45...,Lab custom GRCh38,released,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,ENCFF886MOW,bed narrowPeak,bed,narrowPeak,peaks,GRCh38,ENCSR893NWB,eCLIP,/human-donors/ENCDO000AAC/,EFO:0001187,...,NaN,NaN,NaN,released,s3://encode-public/2016/11/30/ffb7042d-9913-41...,Lab custom GRCh38,released,NaN,NaN,NaN
219,ENCFF507ZJR,bed narrowPeak,bed,narrowPeak,peaks,GRCh38,ENCSR989SMC,eCLIP,/human-donors/ENCDO000AAD/,EFO:0002067,...,NaN,NaN,NaN,released,s3://encode-public/2016/11/30/321fb00b-284d-40...,Lab custom GRCh38,released,mixed read lengths,NaN,NaN
220,ENCFF100OEX,bed narrowPeak,bed,narrowPeak,peaks,GRCh38,ENCSR384KAN,eCLIP,/human-donors/ENCDO000AAC/,EFO:0001187,...,NaN,NaN,NaN,released,s3://encode-public/2016/11/30/633ef9cd-a5d2-43...,Lab custom GRCh38,released,NaN,NaN,NaN
221,ENCFF882APR,bed narrowPeak,bed,narrowPeak,peaks,GRCh38,ENCSR977OXG,eCLIP,/human-donors/ENCDO000AAC/,EFO:0001187,...,NaN,NaN,NaN,released,s3://encode-public/2018/04/29/3cdea817-21b9-4d...,Lab custom GRCh38,released,NaN,NaN,NaN


## Prepare directory structure

In [7]:
BASE_FILE_PATH = Path("./csv/")

# copied from https://stackoverflow.com/a/57892171
def rm_tree(pth: Path):
    for child in pth.iterdir():
        if child.is_file():
            child.unlink()
        else:
            rm_tree(child)
    pth.rmdir()

if BASE_FILE_PATH.exists():
    rm_tree(BASE_FILE_PATH)
    
BASE_FILE_PATH.mkdir()

In [8]:
!rm metadata.tsv

## Transform data

In [9]:
# get all unique protein names - all of our classes
targets = metadata['Experiment target'].unique()
targets

array(['YBX3-human', 'SSB-human', 'PABPN1-human', 'UTP3-human',
       'WDR3-human', 'NIPBL-human', 'WDR43-human', 'EXOSC5-human',
       'UPF1-human', 'APOBEC3C-human', 'WRN-human', 'PABPC4-human',
       'FUS-human', 'SUPV3L1-human', 'ZC3H8-human', 'PHF6-human',
       'FXR2-human', 'FXR1-human', 'TROVE2-human', 'PCBP2-human',
       'SUB1-human', 'SF3B1-human', 'RBFOX2-human', 'AGGF1-human',
       'SDAD1-human', 'HNRNPL-human', 'HNRNPUL1-human', 'KHDRBS1-human',
       'FUBP3-human', 'YWHAG-human', 'FASTKD2-human', 'SND1-human',
       'NONO-human', 'TARDBP-human', 'DDX52-human', 'DDX51-human',
       'STAU2-human', 'BCLAF1-human', 'HLTF-human', 'AQR-human',
       'HNRNPC-human', 'AKAP8L-human', 'DHX30-human', 'AATF-human',
       'DDX6-human', 'SBDS-human', 'LARP4-human', 'DDX55-human',
       'NSUN2-human', 'U2AF2-human', 'METAP2-human', 'LSM11-human',
       'DDX21-human', 'ABCF1-human', 'SLTM-human', 'SUGP2-human',
       'TIAL1-human', 'SMNDC1-human', 'BUD13-human', 'UCHL5-hu

This step takes about 50 minutes on a single core machine.

In [12]:
BED_HEADER = ['chrom', 'chromStart', 'chromEnd', 'name', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'blockCount', 'blockSizes', 'blockStarts']
ALLOWED_CHROMOSOMES = ['chr' + str(i) for i in range(1, 22)] + ['chrX', 'chrY', 'chrMT']

for target in tqdm_notebook(targets):

  protein_name = target[: target.index('-')]
  rows = metadata[metadata['Experiment target'] == target]

  # In general, it is not a good practice to create a DataFrame in loop by appending. 
  # But since we know that we have only one or two rows for these targets, we can use it.
  df = pd.DataFrame([], columns=['chr', 'start', 'end', 'strand'])
  for index, row in rows.iterrows():

    local_file = row['File accession'] + '.bed.gz'

    # downloading files in while-try, because network connection might break sometimes
    success = False
    while not success:
      try:
        request.urlretrieve(row['File download URL'], local_file)
        success = True
      except:
        print('Problem with file ', local_file, '. Trying again.')
    reads = pd.read_csv(local_file, sep='\t', compression='gzip', header=None, names=BED_HEADER)
    
    # keep just necessary columns
    reads = reads[['chrom', 'chromStart', 'chromEnd', 'strand']]
    # keep just sequences from chromosomes chr1 - ch22, chrX, chrX and chrMT
    reads = reads[reads['chrom'].isin(ALLOWED_CHROMOSOMES)]
    # removing outliers - keep just sequences >= 20 bp and <= 100 bp
    reads = reads[((reads['chromEnd'] - reads['chromStart']) >= 20) & ((reads['chromEnd'] - reads['chromStart']) <= 100)]
    # rename columns
    reads.columns = ['chr', 'start', 'end', 'strand']

    # add DataFrame from current file to DataFrame from previous file
    df = pd.concat([df, reads], ignore_index=True)
    
    # cleaning - delete downloaded bed file
    Path(local_file).unlink()

  # add column with name of protein
  df['protein_name'] = protein_name

  filename = protein_name + '.csv.gz'
  df.to_csv(BASE_FILE_PATH / filename, index=False, compression='gzip')

/tmp/ipykernel_21605/2383095856.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for target in tqdm_notebook(targets):


  0%|          | 0/150 [00:00<?, ?it/s]

Problem with file  ENCFF100JBV.bed.gz . Trying again.


## YAML file

In [11]:
# YAML file with metadata
# we store paths of reference fasta files

desc = {
  'version': 0,
  'classes': {}
}
for target in targets:
  name = target[: target.index('-')]
  desc['classes'][name] = {
      'type': 'fa.gz',
      'url': 'http://ftp.ensembl.org/pub/release-97/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz',
      'extra_processing': 'ENSEMBL_HUMAN_GENOME'
  }

with open(BASE_FILE_PATH / 'metadata.yaml', 'w') as fw:    
    yaml.dump(desc, fw)

desc

{'version': 0,
 'classes': {'YBX3': {'type': 'fa.gz',
   'url': 'http://ftp.ensembl.org/pub/release-97/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz',
   'extra_processing': 'ENSEMBL_HUMAN_GENOME'},
  'SSB': {'type': 'fa.gz',
   'url': 'http://ftp.ensembl.org/pub/release-97/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz',
   'extra_processing': 'ENSEMBL_HUMAN_GENOME'},
  'PABPN1': {'type': 'fa.gz',
   'url': 'http://ftp.ensembl.org/pub/release-97/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz',
   'extra_processing': 'ENSEMBL_HUMAN_GENOME'},
  'UTP3': {'type': 'fa.gz',
   'url': 'http://ftp.ensembl.org/pub/release-97/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz',
   'extra_processing': 'ENSEMBL_HUMAN_GENOME'},
  'WDR3': {'type': 'fa.gz',
   'url': 'http://ftp.ensembl.org/pub/release-97/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz',
   'extra_processing': 'ENSEMBL_HUMAN_GENOME'},
  'NIPBL': {'type': 'fa.gz',